<a href="https://colab.research.google.com/github/Alphabf/sas2r/blob/main/SAS_2_R_using_tidyr_for_Data_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SAS_2_R using tidyr for Data Manipulation

| Task                              | R (`tidyr`)                               | SAS Equivalent                                       |
| --------------------------------- | ----------------------------------------- | ---------------------------------------------------- |
| **Pivot wide → long**             | `pivot_longer()`                          | `PROC TRANSPOSE` (wide to long)                      |
| **Pivot long → wide**             | `pivot_wider()`                           | `PROC TRANSPOSE` (long to wide)                      |
| **Separate column**               | `separate(col, into = c(...), sep = ...)` | `SCAN()` or `SUBSTR()` in a `DATA` step              |
| **Unite columns**                 | `unite(new_col, col1, col2, sep = "_")`   | `CATX()` or string concatenation                     |
| **Drop missing rows**             | `drop_na()`                               | `if missing(var) then delete;`                       |
| **Fill missing values**           | `fill()`                                  | `RETAIN` + `IF` logic in `DATA` step                 |
| **Complete missing combinations** | `complete()`                              | Create all combos manually via `MERGE` or `PROC SQL` |
| **Nest data** (list-columns)      | `nest()`                                  | No direct SAS equivalent                             |



| Concept      | `tidyr` in R                      | SAS Tool                            |   |                         |
| ------------ | --------------------------------- | ----------------------------------- | - | ----------------------- |
| Reshaping    | `pivot_longer()`, `pivot_wider()` | `PROC TRANSPOSE`                    |   |                         |
| Column split | `separate()`                      | `SCAN()`, `SUBSTR()` in `DATA` step |   |                         |
| Combine cols | `unite()`                         | `CATX()`, \`                        |   | \` string concatenation |
| Handle NAs   | `drop_na()`, `fill()`             | `if missing()` + `RETAIN` logic     |   |                         |


**1. Reshaping TRANSPOSE**

**SAS**

In [ ]:
data widedata;
    input ID Math Science;
    datalines;
1 90 88
2 85 92
;
run;

proc print; run;

SAS server started using Context SAS Studio compute context with SESSION_ID=fce159c0-d217-4598-b12d-101d8acb13bd-ses0000


Obs,ID,Math,Science
1,1,90,88
2,2,85,92


In [ ]:
proc transpose data=widedata out=longdata(rename=(_NAME_=Subject COL1=Score));
    by ID;
    var Math Science;
run;
proc print; run;

Obs,ID,Subject,Score
1,1,Math,90
2,1,Science,88
3,2,Math,85
4,2,Science,92


in shutdown function


**Transpose to original data**

In [ ]:
proc transpose data=longdata out=wideback(drop=_NAME_);
    by ID;
    id Subject;
    var Score;
run;

proc print data=wideback; run;

Obs,ID,Math,Science
1,1,90,88
2,2,85,92


in shutdown function


**tidyr: pivot_longer()**

In [ ]:
wide_data <- tibble(
  ID = c(1, 2),
  Math = c(90, 85),
  Science = c(88, 92)
)
wide_data

ID,Math,Science
<dbl>,<dbl>,<dbl>
1,90,88
2,85,92


In [ ]:
long_data <- wide_data %>%
  pivot_longer(cols = c(Math, Science), names_to = "Subject", values_to = "Score")
long_data

ID,Subject,Score
<dbl>,<chr>,<dbl>
1,Math,90
1,Science,88
2,Math,85
2,Science,92


**Transpose to original data**

In [ ]:
wideback <- long_data %>%
  pivot_wider(names_from = Subject, values_from = Score)
wideback

ID,Math,Science
<dbl>,<dbl>,<dbl>
1,90,88
2,85,92


**Column split or Separate column**

| Task               | R (tidyr)                                 | SAS                                     |
| ------------------ | ----------------------------------------- | --------------------------------------- |
| Split by delimiter | `separate(FullName, into = ..., sep="_")` | `SCAN(FullName, n, "_")` in `DATA` step |
| Split by position  | `separate(..., sep=5)`                    | `SUBSTR(FullName, start, length)`       |
| Multiple splits    | `separate(..., into = c(...))`            | Multiple `SCAN()` or `SUBSTR()` calls   |


**2. SCANS and substr**

**SAS: SCAN()**

In [ ]:
data students;
    input ID FullName $20.;
    datalines;
1 Alice_Smith
2 Bob_Johnson
;
run;
proc print; run;

Obs,ID,FullName
1,1,Alice_Smith
2,2,Bob_Johnson


In [ ]:
data students_separated;
    set students;
    FirstName = scan(FullName, 1, "_");
    LastName = scan(FullName, 2, "_");
run;
proc print; run;

Obs,ID,FullName,FirstName,LastName
1,1,Alice_Smith,Alice,Smith
2,2,Bob_Johnson,Bob,Johnson


**R: tidyr::separate()**

In [ ]:
library(tidyr)
library(dplyr)

In [ ]:
students <- tibble(
  ID = c(1, 2),
  FullName = c("Alice_Smith", "Bob_Johnson")
)
students

ID,FullName
<dbl>,<chr>
1,Alice_Smith
2,Bob_Johnson


In [ ]:
students_separated <- students %>%
  separate(FullName, into = c("FirstName", "LastName"), sep = "_")
students_separated

ID,FirstName,LastName
<dbl>,<chr>,<chr>
1,Alice,Smith
2,Bob,Johnson


**SAS using SUBSTR()**

In [ ]:
data df;
    input ID Code $8.;
    datalines;
1 ABCD1234
2 WXYZ5678
;
run;
proc print; run;

Obs,ID,Code
1,1,ABCD1234
2,2,WXYZ5678


In [ ]:

data df_split;
    set df;
    Part1 = substr(Code, 1, 4);
    Part2 = substr(Code, 5);
run;
proc print; run;

Obs,ID,Code,Part1,Part2
1,1,ABCD1234,ABCD,1234
2,2,WXYZ5678,WXYZ,5678


**R using tidyr::separate() with sep = 4**

In [ ]:
df <- tibble(
  ID = c(1, 2),
  Code = c("ABCD1234", "WXYZ5678")
)

df

ID,Code
<dbl>,<chr>
1,ABCD1234
2,WXYZ5678


In [ ]:
df_split <- df %>%
  separate(Code, into = c("Part1", "Part2"), sep = 4)
df_split

ID,Part1,Part2
<dbl>,<chr>,<chr>
1,ABCD,1234
2,WXYZ,5678


**3. String concatenation CATX**

**SAS**

In [ ]:
data fullnames;
    input ID FirstName $ LastName $;
    FullName = catx(' ', FirstName, LastName);
    datalines;
1 Alice Smith
2 Bob Johnson
3  Lee
;
run;
proc print; run;

Obs,ID,FirstName,LastName,FullName
1,1,Alice,Smith,Alice Smith
2,2,Bob,Johnson,Bob Johnson


in shutdown function


**Using tidyr::unite()**

In [ ]:
library(tidyr)
library(dplyr)

In [ ]:
df <- tibble(
  ID = c(1, 2, 3),
  FirstName = c("Alice", "Bob", NA),
  LastName = c("Smith", "Johnson", "Lee")
)

df

ID,FirstName,LastName
<dbl>,<chr>,<chr>
1,Alice,Smith
2,Bob,Johnson
3,NA,Lee


In [ ]:
df_united <- df %>%
  unite("FullName", FirstName, LastName, sep = " ", na.rm = TRUE)

df_united

ID,FullName
<dbl>,<chr>
1,Alice Smith
2,Bob Johnson
3,Lee


**Imputing Missing Values**

In [ ]:
data temp;
  input ID Group $ Score;
  datalines;
1 A 10
2 . 15
3 . 12
4 B 20
5 . 18
6 C 30
;
run;

data temp_imputed;
  set temp;
  retain last_group;
  if not missing(Group) then last_group = Group;
  else Group = last_group;
  drop last_group;
run;

proc print data=temp_imputed noobs; run;


ID,Group,Score
1,A,10
2,A,15
3,A,12
4,B,20
5,B,18
6,C,30


**fill() in R**

In [ ]:
install.packages("tidyr")
library(tidyr)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
df <- tibble(
  ID = 1:6,
  Group = c("A", NA, NA, "B", NA, "C"),
  Score = c(10, 15, 12, 20, 18, 30)
)

df %>% fill(Group, .direction = "down")


ID,Group,Score
<int>,<chr>,<dbl>
1,A,10
2,A,15
3,A,12
4,B,20
5,B,18
6,C,30


**Replace missing (NA) values in one or more columns**

**SAS**

In [ ]:
data df;
  input ID Group $ Score;
  datalines;
1 A 10
2 . 15
3 B .
4 . 20
5 C .
;
run;

proc print data=df noobs; run;

SAS server started using Context SAS Studio compute context with SESSION_ID=2d2c8493-4ea9-465d-bb4c-5fb0c13dce3c-ses0000


ID,Group,Score
1,A,10
2,,15
3,B,.
4,,20
5,C,.


In [ ]:
data temp;
  set df;
  if missing(Group) then Group = "Unknown";
  if missing(Score) then Score = 0;
run;
proc print; run;

Obs,ID,Group,Score
1,1,A,10
2,2,Unknown,15
3,3,B,0
4,4,Unknown,20
5,5,C,0


in shutdown function


**replace_na() R**

In [ ]:
install.packages("tidyr")
library(tidyr)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:

df <- tibble(
  ID = 1:5,
  Group = c("A", NA, "B", NA, "C"),
  Score = c(10, 15, NA, 20, NA)
)

temp <- df %>%
  replace_na(list(
    Group = "Unknown",
    Score = 0
  ))

temp

ID,Group,Score
<int>,<chr>,<dbl>
1,A,10
2,Unknown,15
3,B,0
4,Unknown,20
5,C,0
